In [2]:
# import libraries
from utils import * 
from utils_circ_stats import *
import os.path
import sklearn
import glob
import copy
import json
char_labels = task_details[task_details['trial_type']=='Decision'].sort_values(by=['slide_num'])['role_num'].values
char_labels = char_labels[char_labels != 9.0] # exclude neutrals

from scipy import io
from sklearn.feature_selection import VarianceThreshold
import warnings
warnings.filterwarnings("ignore")

In [3]:
def character_task_details():
    
    decision_details = task_details[task_details['trial_type'] == 'Decision']
    decision_details.sort_values('decision_num', inplace=True)
    character_masks = [(decision_details['role']==char.capitalize()).values for char in character_roles]
    
    return [decision_details[character_masks[c]] for c in range(5)]

def character_2d_ixs():
    
    character_details = character_task_details()
    ixs = np.zeros((5,6,2))
    bools = np.zeros((5,12,2)).astype(bool)

    for c in range(5):

        affil_bool   = character_details[c]['dimension'] == 'affil' 
        power_bool   = character_details[c]['dimension'] == 'power' # corret up to here
        bools[c,:,:] = np.concatenate([affil_bool.values.reshape(-1,1), power_bool.values.reshape(-1,1)],1)

        affil_ixs  = character_details[c][affil_bool]['decision_num'].values - 1
        power_ixs  = character_details[c][power_bool]['decision_num'].values - 1
        ixs[c,:,:] = np.concatenate([affil_ixs.reshape(-1,1), power_ixs.reshape(-1,1)],1).astype(int)
        # be careful with how things are concatenated and reshaped! easy to mess up
         
    return ixs.astype(int), bools
         
def get_character_decisions_2d(decisions, padded=True):
    
    # pass in decisions as 1d
    
    char_ixs, char_bools = character_2d_ixs()
    
    if padded:
        character_decisions = np.zeros((5,12,2))
        for c in range(5):
            ixs = char_ixs[c]
            bools = char_bools[c]
            character_decisions[c][:,0][bools[:,0]] = decisions[ixs[:,0]] # affil
            character_decisions[c][:,1][bools[:,1]] = decisions[ixs[:,1]] # power
    else: 
        character_decisions = np.zeros((5,6,2))
        for c in range(5):
            character_decisions[c][:,0] = decisions[char_ixs[c][:,0]] # affil
            character_decisions[c][:,1] = decisions[char_ixs[c][:,1]] # power
            
    return character_decisions 

def get_character_coords(decisions):
    
    if decisions.ndim == 2:
        print('need 63 decisions, not ordered by character')
    character_decisions = get_character_decisions_2d(decisions, padded=True)
    character_coords = np.cumsum(character_decisions, axis=1)
    
    return character_coords

def reshape_char_to_trials(char):
    
    t = 0
    trials = np.zeros((60, 3))
    char_ixs, _ = character_2d_ixs()
    for coords, ixs in zip(char, char_ixs):
        ixs = ixs.flatten()
        ixs.sort()
        trials[t:t+12,:] = np.hstack([coords, ixs.reshape(-1,1)])
        t += 12
        
    return trials[np.argsort(trials[:, 2])][:,0:2]

# behavior

In [4]:
dec_trials = task_details[task_details['trial_type'] == 'Decision']
dims = ['affil', 'power']
characters = ['First', 'Second', 'Assistant', 'Powerful', 'Boss']
metric = 'euclidean'

beh_dir   = '../Behavior/Organized'
beh_files = glob.glob(beh_dir + '/SNT*.xlsx')

all_dvs = {} 

for f in beh_files:
    beh     = pd.read_excel(f)
    sub_id  = f.split('/')[-1].split('_')[1]
    sub_dvs = {}

    # get coordinates...
    curr_xy = {'First': np.array([0,0]),'Second': np.array([0,0]), 'Assistant': np.array([0,0]),'Powerful': np.array([0,0]),'Boss': np.array([0,0])}
    xy = np.zeros((63,2))
    decisions = beh['Decision'].values
    for t in np.arange(0,63):
        char = dec_trials.iloc[t,:]['role']
        if char != 'Neutral':
            d = dims.index(dec_trials.iloc[t,:]['dimension']) 
            curr_xy[char][d] += decisions[t] # add decision    
            xy[t] = copy.deepcopy(curr_xy[char])
    xy_df = pd.DataFrame(np.hstack((np.arange(1,64).reshape(-1,1),decision_details['role_num'].values.reshape(-1,1), xy)), 
                         columns=['decision_trial', 'character_label', 'affil_coord', 'power_coord'])

    #############################################
    # CURRENT TRIAL'S LOCATION CHANGE
    #############################################

    # for a 1d current dimension control
    xy_df['place_1d_coord'] = np.nan
    for dim in ['affil', 'power']: 
        mask = (dec_trials['dimension'] == dim).values
        xy_df['place_1d_coord'][mask] = xy_df[dim + '_coord'][mask]
    xy_df['place_1d_coord'] = xy_df['place_1d_coord'].fillna(0) # neutrals get 0s

    # place distances
    sub_dvs['place_2d_dv'] = get_pw_dist_ut_vec(xy_df.iloc[:,2:4].values, metric=metric)
    sub_dvs['place_1d_dv'] = get_pw_dist_ut_vec(xy_df['place_1d_coord'].values.reshape(-1,1), metric=metric)
    sub_dvs['place_affil_dv'] = get_pw_dist_ut_vec(xy_df['affil_coord'].values.reshape(-1,1), metric=metric)
    sub_dvs['place_power_dv'] = get_pw_dist_ut_vec(xy_df['power_coord'].values.reshape(-1,1), metric=metric)
    sub_dvs['place_positive_dv'] = get_pw_dist_ut_vec((xy_df['affil_coord'].values + xy_df['power_coord'].values).reshape(-1, 1), metric=metric)

    # end trials for decoding
    end_xy = np.array([curr_xy[c] for c in characters]).reshape(-1,2)
    sub_dvs['end_place_2d_dv'] = get_pw_dist_ut_vec(end_xy, metric=metric)
    sub_dvs['end_place_affil_dv']   = get_pw_dist_ut_vec(np.array([curr_xy[c][0] for c in characters]).reshape(-1,1), metric=metric)
    sub_dvs['end_place_power_dv']   = get_pw_dist_ut_vec(np.array([curr_xy[c][1] for c in characters]).reshape(-1,1), metric=metric)
    sub_dvs['end_angle_ori_cos_dv'] = symm_mat_to_ut_vec(pairwise_cosine_distance(end_xy))
    sub_dvs['end_angle_ori_dv'] = symm_mat_to_ut_vec(pairwise_angular_distance(end_xy))
    sub_dvs['end_place_positive_dv'] = get_pw_dist_ut_vec(np.sum(end_xy,axis=1).reshape(-1, 1), metric=metric)

    # vector based
    # referenced from origin
    allo_dists = np.array([vec_norm(v) for v in xy_df.iloc[:,2:4].values])
    sub_dvs['dist_ori_dv'] = get_pw_dist_ut_vec(allo_dists.reshape(-1, 1), metric=metric)
    end_allo_dists = np.array([vec_norm(v) for v in end_xy]) 
    sub_dvs['end_dist_ori_dv'] = get_pw_dist_ut_vec(end_allo_dists.reshape(-1, 1), metric=metric)
    sub_dvs['angle_ori_cos_dv'] = symm_mat_to_ut_vec(pairwise_cosine_distance(xy))
    sub_dvs['angle_ori_dv'] = symm_mat_to_ut_vec(pairwise_angular_distance(xy))
    sub_dvs['end_angle_ori_cos_dv'] = symm_mat_to_ut_vec(pairwise_cosine_distance(end_xy))
    sub_dvs['end_angle_ori_dv'] = symm_mat_to_ut_vec(pairwise_angular_distance(end_xy))

    # referenced from theoretical participant pov
    ego_vectors = np.array([6 - xy_df['affil_coord'].values, 0 - xy_df['power_coord'].values]).T
    ego_dists = np.array([vec_norm(v) for v in ego_vectors]) # checked against matlab preprocessing
    sub_dvs['dist_pov_dv'] = get_pw_dist_ut_vec(ego_dists.reshape(-1, 1), metric=metric)  
    end_ego_vectors = np.array([6 - end_xy[:,0], 0 - end_xy[:,1]]).T
    end_ego_dists = np.array([vec_norm(v) for v in end_ego_vectors]) 
    sub_dvs['end_dist_pov_dv'] = get_pw_dist_ut_vec(end_ego_dists.reshape(-1, 1), metric=metric)

    sub_dvs['angle_pov_cos_dv'] = symm_mat_to_ut_vec(pairwise_cosine_distance_ego(xy))
    sub_dvs['angle_pov_dv'] = symm_mat_to_ut_vec(pairwise_angular_distance_ego(xy))
    sub_dvs['end_angle_pov_cos_dv'] = symm_mat_to_ut_vec(pairwise_cosine_distance_ego(end_xy))
    sub_dvs['end_angle_pov_dv'] = symm_mat_to_ut_vec(pairwise_angular_distance_ego(end_xy))


    #############################################
    # PAST & FUTURE TOO... 
    #############################################

    cd_xchar = get_character_decisions_2d(decisions)
    zeros = np.array([[[0,0]],[[0,0]],[[0,0]],[[0,0]],[[0,0]]])
    nans = np.array([[[np.nan,np.nan]],[[np.nan,np.nan]],[[np.nan,np.nan]],[[np.nan,np.nan]],[[np.nan,np.nan]]])

    # "past" decisions & coordinates: 1-trial back
    pd_xchar = np.hstack([zeros, cd_xchar])[:, 0:12, :]
    pc_xchar = np.cumsum(pd_xchar, 1)
    pc = reshape_char_to_trials(pc_xchar) # 60x2
    pc = add_neutrals(pc) #63x2
    sub_dvs['prev_place_2d_dv'] = get_pw_dist_ut_vec(pc, metric=metric)
    sub_dvs['prev_place_affil_dv'] = get_pw_dist_ut_vec(pc[:,0].reshape(-1,1), metric=metric)
    sub_dvs['prev_place_power_dv'] = get_pw_dist_ut_vec(pc[:,1].reshape(-1,1), metric=metric)
    # can add angle etc...

    # "future" decisions: next trial
    # fc_xchar = np.hstack([cc_xchar, nans])[:,1:,:] # make sure not to cumsum over future decisions only... will change the coordinate values
    # will contain nans so needs more work if want to explore this...

    #############################################
    # CURRENT TRIAL BP, RT ETC...
    #############################################

    # rt & button press dvs                                                     
    sub_dvs['rt_dv'] = symm_mat_to_ut_vec(pairwise_distances(beh['RT (s)'].values.reshape(-1,1), metric=metric))
    sub_dvs['bp_dv'] = symm_mat_to_ut_vec(pairwise_distances(beh['Button_press'].values.reshape(-1,1), metric=metric))

    # non-rdv behavior
    sub_dvs['rt'] = beh['RT (s)'].values
    sub_dvs['bp'] = beh['Button_press'].values
    sub_dvs['decisions'] = beh['Decision'].values
    sub_dvs['coords'] = pd.concat([xy_df['affil_coord'], xy_df['power_coord']], axis=1).values

    # put into sample dictionary
    all_dvs[sub_id] = sub_dvs

with open('../Behavior/Behavioral_distances.json', 'w') as f:
    json.dump(all_dvs, f, cls=json_encoder)

pickle_file(all_dvs,'../Behavior/Behavioral_distances.pkl')                                                

